In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os, sys, random, importlib
import sqlite3
from datetime import date, datetime, timedelta

In [2]:
df = pd.read_csv("dataset.csv")
df.head()

,id,rfid_tag,checkin_time,checkout_time,note,date
0,203628,BA6379D,NaN,NaN,libur,2025-11-23
1,203629,BB39389D,NaN,NaN,libur,2025-11-23
2,203630,AB33A9D,NaN,NaN,libur,2025-11-23
3,203631,ABEA3B9D,NaN,NaN,libur,2025-11-23
4,203632,AB7A3C9D,NaN,NaN,libur,2025-11-23


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159534 entries, 0 to 159533
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159534 non-null  int64 
 1   rfid_tag       159534 non-null  object
 2   checkin_time   66293 non-null   object
 3   checkout_time  40519 non-null   object
 4   note           102503 non-null  object
 5   date           159534 non-null  object
dtypes: int64(1), object(5)
memory usage: 7.3+ MB


In [4]:
def _to_wib_time(val):
    if pd.isna(val):
        return val
    try:
        # coba parse dengan asumsi ada keterangan timezone seperti 'GMT' -> dianggap UTC
        dt = pd.to_datetime(val, utc=True)
        # konversi ke timezone Asia/Jakarta (WIB)
        dt_wib = dt.tz_convert('Asia/Jakarta')
        return dt_wib.strftime('%H:%M:%S')
    except Exception:
        # fallback: parse tanpa tz, lalu tambahkan 7 jam
        try:
            dt = pd.to_datetime(val, errors='coerce')
            if pd.isna(dt):
                return val
            dt_wib = dt + pd.Timedelta(hours=7)
            return dt_wib.strftime('%H:%M:%S')
        except Exception:
            return val

In [5]:
def _to_day_of_week(val):
    if pd.isna(val):
        return val
    try:
        dt = pd.to_datetime(val, errors='coerce')
        if pd.isna(dt):
            return val
        return dt.day_name()
    except Exception:
        return val

In [6]:
# terapkan ke kolom (biarkan missing values seperti semula)
df['checkin_time'] = df['checkin_time'].apply(_to_wib_time)
df['checkout_time'] = df['checkout_time'].apply(_to_wib_time)

In [7]:
# terapkan ke kolom 'date' tanpa mengubah missing values
df['DayOfWeek'] = df['date'].apply(_to_day_of_week)

In [8]:
# isi missing di kolom 'note' berdasarkan ada/tidaknya 'checkin_time'
mask_missing = df['note'].isna() | (df['note'].astype(str).str.strip() == '')

# jika checkin_time ada -> hadir, jika tidak ada -> alpa
df.loc[mask_missing & df['checkin_time'].notna(), 'note'] = 'hadir'
df.loc[mask_missing & df['checkin_time'].isna(), 'note'] = 'alpa'

# optional: cek hasil singkat
df['note'].value_counts(dropna=False).head()

note
hadir    57031
libur    47011
alpa     46231
telat     9261
Name: count, dtype: int64

In [9]:
def _to_minutes(x):
    if pd.isna(x):
        return np.nan
    if isinstance(x, (int, float)):
        return float(x)
    if isinstance(x, str):
        # accept "HH:MM" or "HH:MM:SS"
        try:
            t = datetime.strptime(x.strip(), "%H:%M:%S").time()
        except Exception:
            try:
                t = datetime.strptime(x.strip(), "%H:%M").time()
            except Exception:
                # fallback: numeric string
                try:
                    return float(x)
                except Exception:
                    return np.nan
        return t.hour * 60 + t.minute + t.second / 60.0
    if isinstance(x, time):
        return x.hour * 60 + x.minute + x.second / 60.0
    return np.nan

def add_temporal_and_lag_features(
    df,
    student_id_col='StudentID',
    date_col='Date',
    arrival_col='ArrivalTime',
    status_col='Status',
    late_labels=None,
    absent_labels=None
):
    """
    Adds:
      - Lag_1_Status
      - Count_Telat_7D (count of late in prior 7 days)
      - Count_Alpa_30D (count of absent in prior 30 days)
      - Streak_Telat (consecutive late days up to t-1)
      - Avg_Arrival_Time_7D (minutes, prior 7 days)
    Assumes df has one row per student per date. Returns new DataFrame with added columns.
    """
    if late_labels is None:
        late_labels = {'Telat', 'telat', 'Late', 'late'}
    if absent_labels is None:
        absent_labels = {'Alpa', 'alpa', 'Absent', 'absent', 'ALPA', 'ALP'}
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    # prepare arrival minutes
    df['_arrival_min'] = df[arrival_col].map(_to_minutes)

    groups = []
    for _, g in df.groupby(student_id_col, sort=False):
        g = g.sort_values(date_col).copy()
        g.set_index(date_col, inplace=True)

        is_late = g[status_col].isin(late_labels).astype(int)
        is_absent = g[status_col].isin(absent_labels).astype(int)
        arr_min = g['_arrival_min'].astype(float)

        # Lag 1 status (previous row for same student)
        lag_status = g[status_col].shift(1)

        # Count late in prior 7 days (exclude today)
        count_telat_7d = is_late.rolling('7d', closed='left').sum().fillna(0).astype(int)

        # Count absent in prior 30 days (exclude today)
        count_alpa_30d = is_absent.rolling('30d', closed='left').sum().fillna(0).astype(int)

        # Streak of consecutive late days up to previous day.
        # compute run lengths of is_late; streak_including counts current day when late.
        run_group = (is_late == 0).cumsum()
        streak_including = is_late.groupby(run_group).cumcount() + 1
        streak_including = streak_including.where(is_late == 1, 0)
        streak_prev = streak_including.shift(1).fillna(0).astype(int)

        # Avg arrival time in prior 7 days (minutes), exclude today
        avg_arrival_7d = arr_min.rolling('7d', closed='left').mean()

        g['Lag_1_Status'] = lag_status.values
        g['Count_Telat_7D'] = count_telat_7d.values
        g['Count_Alpa_30D'] = count_alpa_30d.values
        g['Streak_Telat'] = streak_prev.values
        g['Avg_Arrival_Time_7D'] = avg_arrival_7d.values

        groups.append(g.reset_index())

    result = pd.concat(groups, axis=0, ignore_index=True)
    result = result.sort_values([student_id_col, date_col]).reset_index(drop=True)
    result.drop(columns=['_arrival_min'], inplace=True, errors='ignore')
    return result

In [10]:
enhanced = add_temporal_and_lag_features(df, student_id_col='rfid_tag',
                                            date_col='date',
                                            arrival_col='checkin_time',
                                            status_col='note')
enhanced.to_csv("attendance_with_features.csv", index=False)

In [11]:
featured_df = pd.read_csv("attendance_with_features.csv")
featured_df.head()

,date,id,rfid_tag,checkin_time,checkout_time,note,DayOfWeek,Lag_1_Status,Count_Telat_7D,Count_Alpa_30D,Streak_Telat,Avg_Arrival_Time_7D
0,2025-09-10,94907,1418C9BC,14:39:32,14:39:40,telat,Wednesday,NaN,0,0,0,NaN
1,2025-09-11,96391,1418C9BC,NaN,NaN,alpa,Thursday,telat,1,0,1,879.533333
2,2025-09-12,97876,1418C9BC,07:22:42,13:50:34,telat,Friday,alpa,1,1,0,879.533333
3,2025-09-13,99361,1418C9BC,NaN,NaN,libur,Saturday,telat,2,1,2,661.116667
4,2025-09-14,100846,1418C9BC,NaN,NaN,libur,Sunday,libur,2,1,0,661.116667
